<div style="width:100%; text-align:center; margin: 40px 0;">
    <h1 style="color: #003366; font-size: 40px; font-weight:bold; text-shadow: 2px 2px 5px grey;">
        Énergie Électrique en France
    </h1>
    <hr style="width: 50%; height: 3px; background-color: #003366; border: none;">
    <p style="font-style:italic; font-size:18px; color: #555;">
        Une exploration approfondie de l'énergie électrique en France
    </p>
</div>


## Données:

Dans ce projet nous allons travailler sur des données produites par RTE France. Nous allons récolter ces données grâce à une API (OpenData Reseaux Energie https://opendata.reseaux-energies.fr/). Nous allons travailler sur des données régionales récoltées au niveau régional et sur des données nationales. Nous aurons aussi des API qui contiennent des données en temps réel, ce qui nous permettra d'avoir des données mise à jour régulierement.  

Voici les 4 liens API que nous utiliserons durant ce projet:
- Données Nationales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-tr`
- Données Nationales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-national-cons-def`
- Données Régionales en temps réel (du 1er Juin 2022 à aujourd'hui): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-tr`
- Données Régionales fixes (jusqu'au 31 Mai 2022): `https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/eco2mix-regional-cons-def`

Grâce au module `request` de python nous allons pouvoir faire des requêtes de type SQL à ces API dans le but d'obtenir les données dont nous avons besoin au format JSON.

Nous utiliserons aussi un fichier GeoJson récupéré sur le site `https://france-geojson.gregoiredavid.fr/`. Ce fichier possède les données géographiques de la France, par région ou par département. Les données de RTE France étant uniquement nationales et régionales, nous utiliserons uniquement le GeoJson des régions de la France.

### Variables que nous allons analyser

- **Date** : *Date à laquelle les données ont ete enregistrees*
- **Heure** : *Date à laquelle les données ont ete enregistrees*
- **Région** (pour les données régionales) : *La région correspondant aux données*
- **Consommation** (MW) : *La consommation électrique réelle*
- **Prévision**  Conso J et J-1 (MW) :  *La consommation électrique prédite par RTE France au jours J et J-1*
- **Production par filières** (MW) : *La production d'energie electrique pqr filiere (nucleaire, hydrolique, eolien, solaire...)*
- **Imports exports  avec plusieurs pays** (MW) : *Echanges d'energie avec les differents pays frontaliers (Suisse, Allemagne, Belgique, Angleterre, Espagne)*
- **Emission de CO2** (g/kWh) : *La valeur d'emission de CO2*

## Type d'analyse prévue

Nous allons effectuer plusieurs visualisations sur nos données afin d'en tirer des observations. Pour cela nous allons principalement utiliser le module `plotly express` qui permet de faire des graphiques interactifs. Nous allons également améliorer l'interactivitée de notre graphique en rajoutant des modules IPython permetant à l'utilisateur de pouvoir trier et filtrer les données.  
Nous effecturons également des analyses statistiques afin de repondre à certains questionnements sur l'électricitée en France.

Pour cela nous concentrerons notre analyse sur 3 axes principaux:
- La consommation électrique en France
- La production d'énergie électrique aux niveaux national et régional
- Les échanges commerciaux avec les pays frontaliers

Nous allons d'abord devoir récupérer, nettoyer et formater les données lors des requêtes à l'API d'OpenData. Nous utiliserons ensuite Python et les bibliothèques Pandas et Plotly pour manipuler ces données et créer des visualisations.

## Résultats attendus

A la fin de cette analyse, nous devrions être capable de répondre à plusieurs questions tel que:
- Quels sont les modes de production les plus importants en France ? Par régions ?
- Comment a évolué la part du renouvelable dans le mix énergétique français durant la dernière décennie ?
- Quel est la balance commerciale de la France pour l’énergie ?
- Comment évolue la production et la consommation de l’électricité  en France  durant 1 journée ? 1 mois ? 1 année ?

Voici le type de graphique que l'on pourrait avoir à la fin de notre analyse:  

Voici un graphique représentant la consommation d'énergie électrique en France sur une seule journée.

![Consommation électrique](IMG/conso.png)

Ensuite nous pourrions avoir un graphique qui montre les échanges d'électricité avec les pays frontaliers.  
  
![Echanges commerciaux](IMG/echanges_comm.png)

Ensuite voici un graphique qui montre la proction d'energie électrique par la filière solaire sur une journée. Nous pouvons voir que le pic de production se situe vers 12h ce qui correspond au moment ou il y a le plus de soleil dans la journée.  
  
![Production d'energie solaire](IMG/production_solaire.png)  

Et pour finir voici un graphique representant la part de production d'énergie électrique pour chaque filiere:  
  
![Production d'energie par filiere](IMG/pie_chart_production.png)  


In [65]:
# Data

URL = "https://odre.opendatasoft.com/api/explore/v2.1/catalog/datasets/"

# Imports
import requests
import json

import pandas as pd

import plotly.express as px


In [61]:
def fetch_data_by_date(data:str, start:int, rows:int, date:str):
    """Fetch data from a dataset by date and offset
    
    Parameters
    ----------
    data : str
        Name of the dataset.
    start : int
        Offset.
    rows : int
        Number of rows.
    date : str
        Date of the data.

    Returns
    -------
    dict
        Dictionary containing the data. 
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "offset" : start,
        "rows": rows,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('results', {})
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return 
    
def get_length_per_date(data:str, date:str):
    """Get the number of rows for a given date

    Parameters
    ----------
    data : str
        Name of the dataset.
    date : str
        Date of the data.

    Returns
    -------
    int
        Number of rows.
    """
    url = f"{URL}{data}" + "/records"
    params = {
        "select": "date",
        "rows": 1,
        "where": f"date='{date}'"
    }
    response = requests.get(url, params=params)
    data=[{}]
    if response.status_code == 200:
        data = response.json()
        return data.get('total_count')
    else:
        print(f"Échec de la requête: {response.status_code}")
        print(response.text)
        return
      

In [62]:
def insert_data_into_json(file_name: str, new_data: list):
    """Insère de nouvelles données dans un fichier JSON existant.

    Parameters
    ----------
    file_name : str
        Le nom du fichier JSON.
    new_data : list
        La liste des nouvelles données à insérer.
    """
    try:
        with open(file_name, 'r') as file:
            existing_data = json.load(file)
    except FileNotFoundError:
        existing_data = []

    existing_data.extend(new_data)

    with open(file_name, 'w') as file:
        json.dump(existing_data, file)

new_data = fetch_data_by_date('eco2mix-national-tr', 0, 96, '2023-01-01')
insert_data_into_json('combined_data.json', new_data)


In [69]:
def transform_json_to_df(file_name:str):
    """Transform a Json File to a pandas DataFrame"""
    with open(file_name) as f:
        data = json.load(f)
    return pd.DataFrame(data)
    
df = transform_json_to_df('combined_data.json')
df

,perimetre,nature,date,heure,date_heure,consommation,prevision_j1,prevision_j,fioul,charbon,...,gaz_ccg,gaz_autres,hydraulique_fil_eau_eclusee,hydraulique_lacs,hydraulique_step_turbinage,bioenergies_dechets,bioenergies_biomasse,bioenergies_biogaz,stockage_batterie,destockage_batterie
0,France,Données temps réel,2023-01-01,01:00,2023-01-01T00:00:00+00:00,45197,43800,45700,204,44,...,420,0,4854,676,922,133,427,257,ND,ND
1,France,Données temps réel,2023-01-01,01:15,2023-01-01T00:15:00+00:00,45675,44100,45950,204,44,...,416,0,4858,659,914,136,425,257,ND,ND
2,France,Données temps réel,2023-01-01,03:00,2023-01-01T02:00:00+00:00,42697,41200,43200,202,44,...,418,0,4774,0,414,137,423,257,ND,ND
3,France,Données temps réel,2023-01-01,04:00,2023-01-01T03:00:00+00:00,39975,38300,40500,202,44,...,550,0,4652,0,226,138,422,257,ND,ND
4,France,Données temps réel,2023-01-01,08:30,2023-01-01T07:30:00+00:00,39263,36200,39300,204,46,...,438,0,4662,0,0,134,424,257,ND,ND
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,France,Données temps réel,2023-01-01,18:00,2023-01-01T17:00:00+00:00,46315,44900,45900,200,50,...,622,0,5423,2165,2232,111,426,257,ND,ND
92,France,Données temps réel,2023-01-01,20:45,2023-01-01T19:45:00+00:00,45950,45350,46050,200,56,...,454,0,5342,896,345,119,429,257,ND,ND
93,France,Données temps réel,2023-01-01,22:30,2023-01-01T21:30:00+00:00,44463,43700,44200,200,55,...,412,0,4959,797,488,123,432,257,ND,ND
94,France,Données temps réel,2023-01-01,23:00,2023-01-01T22:00:00+00:00,45451,45800,46300,200,56,...,412,0,4930,786,578,121,431,257,ND,ND


In [71]:
## Creation d'un premier graphique plotly

fig = px.line(df, x="heure", y="consommation", title='Consommation électrique en France le 1er janvier 2023')

fig.show()